In [1]:
import pandas as pd

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

In [2]:
input_ = pd.read_excel('./1.menu.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['No'] = input_['No'].astype(int)

print(f'总数量：{len(input_)}')
input_['Item Number'].tolist()

总数量：10


['110940194723',
 '112296912635',
 '113134507797',
 '114410510230',
 '115186863085',
 '115186863204',
 '115244844718',
 '115244856836',
 '115244861897',
 '115244875952']

In [3]:
a = 0

a

0

In [4]:
crawler_status = 'error'

crawler_status

'error'

In [5]:
import requests

import sys
sys.path.append('../../00.Tools')
from crawler_configuration import get_header, get_proxy

b = 0
while True:
    b += 1
    print(b)
    try:
        resp = requests.get(input_.loc[a, 'Url'],
                            headers=get_header(ua=False),
                            proxies=get_proxy(),
                            timeout=(10, 10))

        if resp.status_code == 200 or resp.status_code == 404:
            break
    except:
        continue

resp

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104


<Response [200]>

In [6]:
resp.status_code == 404

False

In [7]:
import json

json_ = json.loads(resp.text.split('$MC=(window.$MC||[]).concat(')[1].split(']}]})</script>')[0] + ']}]}')

json_

{'o': {'l': 1,
  'g': {'enableATCRedesign': 'false',
   'lazyLoaderType': False,
   'largerImageSize': 'FIXED_15',
   'signedIn': False,
   'epInfo': {'featureFlagDetails': {'dwebDeliSSE': {'value': {'NORI_ENABLED': 'true'},
      'dropTag': 'server'},
     'enableSMEMerchBTF': {'value': False},
     'dwebAtfStreamingFeature': {'value': {'NORI_ENABLED': 'true',
       'DELI_ENABLED': 'false'}},
     'prefetchResourceFlag': {'value': {'VINODE_ENABLE_PREFETCH_RESOURCES': 'false'}},
     'dwebAtfSplitFeature': {'value': None},
     'enableOTPFeature': {'value': {'enableOTPFlow': 'false',
       'enableGuestOTPAuthFlow': False},
      'dropTag': 'browser'},
     'enableATCRedesign': {'value': {'enableATCLayerV3': 'false'}},
     'BXR_BUNDLED_TEST': {'value': {'ADS_SME_BTF_ENABLED': 'true',
       'NORI_ENABLED': 'false',
       'DELI_ENABLED': 'true'}},
     'lazyLoaderType': {'value': False}},
    'qTags': {}},
   'dWebATFSplit': 'DEFAULT',
   'renderProgressively': True,
   'isPrefetchRe

In [8]:
if input_.loc[a, 'Url'].startswith('https://www.ebay.com/'):
    market, country, language, locale = 'EBAY-US', 'US', 'en', 'en-US'
elif input_.loc[a, 'Url'].startswith('https://www.ebay.de/'):
    market, country, language, locale = 'EBAY-DE', 'DE', 'de', 'de-DE'
elif input_.loc[a, 'Url'].startswith('https://www.ebay.co.uk/'):
    market, country, language, locale = 'EBAY-GB', 'GB', 'en', 'en-GB'
elif input_.loc[a, 'Url'].startswith('https://www.ebay.com.au/'):
    market, country, language, locale = 'EBAY-AU', 'AU', 'en', 'en-AU'

print(market, country, language, locale)

json_['o']['g']['marketplaceId'].strip() != market or json_['o']['g']['country'].strip() != country or json_['o']['g']['language'].strip() != language or json_['o']['g']['locale'].strip() != locale or json_['o']['g']['itemId'].strip() != input_.loc[a, 'Item Number']

EBAY-US US en en-US


False

In [9]:
index_model = -1
for i in range(len(json_['o']['w'])):
    if 'model' in json_['o']['w'][i][2]:
        index_model = i
        break

index_model

0

In [10]:
index_model == -1

False

In [11]:
compatibility = json_['o']['w'][index_model][2]['model']['modules']['COMPATIBILITY_TABLE']['paginatedTable']['pagination']['pages'][-1] if 'COMPATIBILITY_TABLE' in json_['o']['w'][index_model][2]['model']['modules'] else ''
dict_compatibility = {} if compatibility == '' else {'api': compatibility['action']['URL'],
                                                     'page': int(compatibility['text'].strip()),
                                                     'data': {'scopedContext': {'catalogDetails': {'categoryId': compatibility['action']['params']['categoryId'],
                                                                                                   'itemId': compatibility['action']['params']['itemId'],
                                                                                                   'marketplaceId': compatibility['action']['params']['marketplaceId'],
                                                                                                   'sellerName': compatibility['action']['params']['sellerName']}}}}

dict_compatibility

{'api': 'https://www.ebay.com/g/api/finders',
 'page': 9,
 'data': {'scopedContext': {'catalogDetails': {'categoryId': '262183.33692',
    'itemId': '110940194723',
    'marketplaceId': 'EBAY-US',
    'sellerName': 'zzperformance_esales'}}}}

In [12]:
dict_specific = {}
dict_item = json_['o']['w'][index_model][2]['model']['modules']['ABOUT_THIS_ITEM']['sections']['features']['dataItems']
for i, (key, value) in zip(range(len(dict_item.items())), dict_item.items()):
    dict_specific[str(i)] = {value['labels'][0]['textSpans'][0]['text'].strip(): value['values'][0]['textualDisplays'][0]['textSpans'][0]['text'].strip() if 'textualDisplays' in value['values'][0] else value['values'][0]['textSpans'][0]['text'].strip()}

dict_specific

{'0': {'Condition': "New: A brand-new, unused, unopened, undamaged item in its original packaging (where packaging is applicable). Packaging should be the same as what is found in a retail store, unless the item was packaged by the manufacturer in non-retail packaging, such as an unprinted box or plastic bag. See the seller's listing for full details."},
 '1': {'Interchange Part Number': '10467067, 10468391, 10472401, 10477602, 10482928, ZZ-HVCP-38-V2, ZZ-SPW-AB-B, ZZ-SPW-AB-BLUE, ZZ-SPW-AB-R'},
 '2': {'Bundle Listing': 'Yes'},
 '3': {'Manufacturer Part Number': 'ZZ-SPW-AB-R/WC'},
 '4': {'Other Part Number': 'D555'},
 '5': {'Brand': 'ZZPerformance'},
 '6': {'Fitment Type': 'Direct Replacement'},
 '7': {'Warranty': '2 Year'},
 '8': {'UPC': 'Does Not Apply'}}

In [13]:
description_url = json_['o']['w'][index_model][2]['model']['modules']['ITEM_DESCRIPTION_MIN_VIEW_MODULE']['sections'][0]['dataItems'][0]['action']['URL'].strip() if 'dataItems' in json_['o']['w'][index_model][2]['model']['modules']['ITEM_DESCRIPTION_MIN_VIEW_MODULE']['sections'][0] else json_['o']['w'][index_model][2]['model']['modules']['ITEM_DESCRIPTION_MIN_VIEW_MODULE']['sections'][0]['action']['URL'].strip()

description_url

'https://vi.vipr.ebaydesc.com/itmdesc/110940194723?t=1594671746000&category=262183&seller=zzperformance_esales&excSoj=1&excTrk=1&lsite=100&ittenable=true&domain=ebay.com&descgauge=1&cspheader=1&oneClk=2&secureDesc=1'

In [14]:
title = json_['o']['w'][index_model][2]['model']['modules']['TITLE']['mainTitle']['textSpans'][0]['text'].strip()

title

'ZZPerformance 3800 High Voltage Coil Packs + 10.5mm Red Spark Plug Wire Combo'

In [15]:
subtitle = json_['o']['w'][index_model][2]['model']['modules']['TITLE']['subTitle']['textSpans'][0]['text'].strip() if 'subTitle' in json_['o']['w'][index_model][2]['model']['modules']['TITLE'] else ''

subtitle

''

In [16]:
price = json_['o']['w'][index_model][2]['model']['modules']['BUY_BOX']['binModel']['price']['textSpans'][0]['text'].strip()
# price = json_['o']['w'][0][2]['model']['modules']['BUY_BOX']['bidPrice']['textSpans'][0]['text'].strip()

price

'US $179.99'

In [17]:
available, sold = '', ''
list_quantity = json_['o']['w'][index_model][2]['model']['modules']['QUANTITY']['availabilitySignal']['textSpans'] if 'QUANTITY' in json_['o']['w'][index_model][2]['model']['modules'] else []
for quantity in list_quantity:
    if input_.loc[a, 'Url'].startswith('https://www.ebay.de/'):
        if 'verfügbar' in quantity['text']:
            available = quantity['text'].strip()
        elif 'verkauft' in quantity['text']:
            sold = quantity['text'].strip()
    else:
        if 'available' in quantity['text']:
            available = quantity['text'].strip()
        elif 'sold' in quantity['text']:
            sold = quantity['text'].strip()

available, sold

('More than 10 available', '258 sold')

In [18]:
dict_src = {}
list_media = json_['o']['w'][index_model][2]['model']['modules']['PICTURE']['mediaList'] if 'mediaList' in json_['o']['w'][index_model][2]['model']['modules']['PICTURE'] else []
for i, media in zip(range(len(list_media)), list_media):
    dict_src[str(i)] = media['image']['originalImg']['URL'].strip()

dict_src

{'0': 'https://i.ebayimg.com/images/g/PDAAAOSwHHRa6hmJ/s-l500.jpg'}

In [19]:
df_temp = pd.DataFrame([{'No': input_.loc[a, 'No'],
                         'Item Number': input_.loc[a, 'Item Number'],
                         'Title': title,
                         'Subtitle': subtitle,
                         'Price': price,
                         'Available': available,
                         'Sold': sold,
                         'Picture': '',
                         'Url': input_.loc[a, 'Url'],
                         'Description Url': description_url,
                         'Json_Src': json.dumps(dict_src),
                         'Json_Specific': json.dumps(dict_specific),
                         'Json_Compatibility': json.dumps(dict_compatibility)}])

output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

output_correct

,No,Item Number,Title,Subtitle,Price,Available,Sold,Picture,Url,Description Url,Json_Src,Json_Specific,Json_Compatibility
0,1,110940194723,ZZPerformance 3800 High Voltage Coil Packs + 1...,,US $179.99,More than 10 available,258 sold,,https://www.ebay.com/itm/110940194723?_ul=US&_...,https://vi.vipr.ebaydesc.com/itmdesc/110940194...,"{""0"": ""https://i.ebayimg.com/images/g/PDAAAOSw...","{""0"": {""Condition"": ""New: A brand-new, unused,...","{""api"": ""https://www.ebay.com/g/api/finders"", ..."


In [20]:
crawler_status = 'ok'

crawler_status

'ok'

In [21]:
if crawler_status == 'error':
    df_temp = pd.DataFrame([{'No': input_.loc[a, 'No'],
                             'Item Number': input_.loc[a, 'Item Number'],
                             'Url': input_.loc[a, 'Url']}])

    output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

output_error

""


In [22]:
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['No'],
                                                ascending=[True],
                                                ignore_index=True)
    output_correct.to_excel('./test_item.xlsx', index=False)

output_correct

,No,Item Number,Title,Subtitle,Price,Available,Sold,Picture,Url,Description Url,Json_Src,Json_Specific,Json_Compatibility
0,1,110940194723,ZZPerformance 3800 High Voltage Coil Packs + 1...,,US $179.99,More than 10 available,258 sold,,https://www.ebay.com/itm/110940194723?_ul=US&_...,https://vi.vipr.ebaydesc.com/itmdesc/110940194...,"{""0"": ""https://i.ebayimg.com/images/g/PDAAAOSw...","{""0"": {""Condition"": ""New: A brand-new, unused,...","{""api"": ""https://www.ebay.com/g/api/finders"", ..."


In [23]:
if not output_error.empty:
    output_error = output_error.sort_values(by=['No'],
                                            ascending=[True],
                                            ignore_index=True)
    output_error.to_excel('./test_item_error.xlsx', index=False)

output_error

""
